# Introduction to Bedrock - Building with Bedrock Embeddings

In [ ]:
#!unzip ../bedrock-preview-documentation/SDK/bedrock-python-sdk.zip -d /root/bedrock

#!pip install --upgrade pip
#!pip install scikit-learn seaborn

#!pwd
#!python3 -m pip install /root/bedrock/boto3-1.26.142-py3-none-any.whl
#!python3 -m pip install /root/bedrock/botocore-1.29.142-py3-none-any.whl

In [14]:
!pwd
!python3 -m pip install boto3-1.26.142-py3-none-any.whl --quiet
!python3 -m pip install botocore-1.29.142-py3-none-any.whl --quiet

/root/bedrock-python-sdk

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [20]:
import boto3
import json
import csv
from datetime import datetime

bedrock = boto3.client(
 service_name='bedrock',
 region_name='us-east-1',
 endpoint_url='https://bedrock.us-east-1.amazonaws.com'
)

In [21]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '422f61fc-41f6-45a8-a688-68d8dfecfa1c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 07 Jun 2023 20:18:57 GMT',
   'content-type': 'application/json',
   'content-length': '861',
   'connection': 'keep-alive',
   'x-amzn-requestid': '422f61fc-41f6-45a8-a688-68d8dfecfa1c'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-e1t-medium',
   'modelId': 'amazon.titan-e1t-medium'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl',
   'modelId': 'stability.stable-diffusion-xl'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-grande-instruct',
   'modelId': 'ai21.j2-grande-instruct'},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/ai21.j2-jumbo-instruct',
   'modelId': 'ai21.j2-jumbo-in

In [74]:
def call_bedrock(modelId, prompt_data):
    if 'amazon' in modelId:
        body = json.dumps({"inputText": prompt_data})
        #modelId = 'amazon.titan-tg1-large'
    elif 'anthropic' in modelId:
        body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 4096})
        #modelId = 'anthropic.claude-instant-v1'
    elif 'ai21' in modelId:
        body = json.dumps({"prompt": prompt_data,
            "maxTokens":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        })
    elif 'stability' in modelId:
        body = json.dumps({"text_prompts":[{"text":prompt_data}]}) 
        #modelId = 'stability.stable-diffusion-xl'
    else:
        print('Parameter model must be one of titan, claude, j2, or sd')
        return
    accept = 'application/json'
    contentType = 'application/json'

    before = datetime.now()
    response = bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    latency = (datetime.now() - before)
    response_body = json.loads(response.get('body').read())

    if 'amazon' in modelId:
        response = response_body.get('results')[0].get('outputText')
    elif 'anthropic' in modelId:
        response = response_body.get('completion')
    elif 'ai21' in modelId:
        response = response_body.get('completions')[0].get('data').get('text')

    #Add interaction to the local CSV file...
    #column_name = ["timestamp", "modelId", "prompt", "response", "latency"] #The name of the columns
    data = [datetime.now(), modelId, prompt_data, response, latency] #the data
    with open('prompt-data.csv', 'a') as f:
        writer = csv.writer(f)
        #writer.writerow(column_name)
        writer.writerow(data)
    
    return response, latency

### Titan

In [29]:
prompt_data = """
Generate a list of reasons why customers should have a Medicine Ball at home using the following data
Weighted medicine ball for upper, lower, and full body exercises
The ball has a lightly textured surface provides a superior grip
Weight clearly labeled on both sides.
Ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more
The person tool to help you develop core strength, balance, and coordination.

"""

In [30]:
response, latency = call_bedrock('amazon.titan-tg1-large', prompt_data)
print(response, "\n\n", "Inference time:", latency)

Answer:
Customers should have a weighted medicine ball at home for the following reasons: 
1. It can be used for upper, lower, and full-body exercises.
2. The lightly textured surface provides a superior grip.
3. We can easily identify the weight of the ball as it is clearly labeled on both sides.
4. It helps to develop core strength, balance, and coordination. 

 Inference time: 0:00:02.822809


### Claude

In [97]:
prompt_data = """Human: Generate a list of reasons why customers should have a Medicine Ball at home using the data inside <metadata></metadata> XML tags.

<metadata>
Weighted medicine ball for upper, lower, and full body exercises
The ball has a lightly textured surface provides a superior grip
Weight clearly labeled on both sides.
Ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more
The person tool to help you develop core strength, balance, and coordination.
</metadata>

Asistant:
"""

In [98]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

Here is a list of reasons why customers should have a Medicine Ball at home:

•It provides a weighted tool for upper, lower, and full body exercises. 
•It has a textured surface that provides a good grip.
•The weight is clearly labeled on both sides so you know how heavy it is.
•It can be used for classic medicine ball workouts like ball tosses, weighted twists, squats, sit ups, and more.
•It helps develop core strength, balance, and coordination. 

 Inference time: 0:00:02.553383


### Jurassic-2

In [32]:
prompt_data ="""Generate a list of reasons why customers should have a Medicine Ball at home using the following data: Weighted medicine ball for upper, lower, and full body exercises
The ball has a lightly textured surface provides a superior grip. Weight clearly labeled on both sides. Ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more. The person tool to help you develop core strength, balance, and coordination."""

In [31]:
prompt_data = """
Generate a description for a red t-shirt

"""

In [54]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data)
print(response, "\n\n", "Inference time:", latency)

Here's a description for a red t-shirt:
The t-shirt is red, made from a soft and comfortable fabric, and fits snugly. 

 Inference time: 0:00:00.817721


In [32]:
response, latency = call_bedrock('amazon.titan-tg1-large', prompt_data)
print(response, "\n\n", "Inference time:", latency)

A simple red t-shirt that can be worn as part of a uniform or paired with jeans. It is made of soft cotton and fits comfortably.  

 Inference time: 0:00:01.520414


In [33]:
prompt_data = """
Human: Generate a description for a red t-shirt
Assistant:
"""

In [34]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)

 Here is a possible description for a red t-shirt:

Red cotton crewneck t-shirt. Short sleeves. Round neckline. Solid bright red color. Made of 100% cotton for a soft and breathable feel. Straight cut with a relaxed, comfortable fit. Versatile staple that can be dressed up or down and paired with jeans, shorts or khakis. 

 Inference time: 0:00:02.538685


In [35]:
prompt_data = """
Human: Generate a description for a red t-shirt
Assistant:
"""

In [36]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data)
print(response, "\n\n", "Inference time:", latency)

A red t-shirt with a round neck and short sleeves, made from 100% cotton. 

 Inference time: 0:00:00.538712


In [131]:
prompt_data = """
Complete the following instructions in order
Intructions:
1. Generate a description for red t-shirt
2. Createa description for a blue hat
3. Generate one reason why people should wear a blue hat

"""

In [132]:
response, latency = call_bedrock('amazon.titan-tg1-large', prompt_data)
print(response, "\n\n", "Inference time:", latency)

1. Generate a description for red t-shirt
Red t-shirt is made of 100% cotton. It is a product of fine cotton which is highly breathable and comfortable. 
2. Createa description for a blue hat
Blue hat is a classic accessory that can match with different styles of outfits. It is made of 100% cotton and it is adjustable. 
3. Generate one reason why people should wear a blue hat
People should wear a blue hat because it can protect them from the sun. The blue color of the hat can also make them look more stylish. 

 Inference time: 0:00:03.834686


In [139]:
prompt_data = """Human: Follow the instruction inside <metadata></metadata> XML tags.

<metadata>
1. Generate a description for red t-shirt
2. Create description for a blue hat
3. Generate one reason why people should wear a blue hat
</metadata>

Asistant:
"""

In [140]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)


1. A red cotton t-shirt with short sleeves, round neckline and a relaxed fit.

2. A navy blue baseball cap made of cotton with an adjustable strap closure.

3. A blue hat can help protect your head and face from the sun's UV rays. 

 Inference time: 0:00:02.016885


In [141]:
prompt_data ="""Execute the following instructions: 
1. Generate a description for red t-shirt
2. Create description for a blue hat
3. Generate one reason why people should wear a blue hat
"""

In [142]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data)
print(response, "\n\n", "Inference time:", latency)

1. A bright red t-shirt with a round neck and comfortable fit.
2. A bright blue hat with a curved brim, adjustable strap, and breathable fabric.
3. Wearing a blue hat can protect your head from the sun and keep you cool. 

 Inference time: 0:00:01.282441


In [43]:
prompt_data = """
Summarize the following text:
Weighted medicine ball for upper, lower, and full body exercises. The ball has a lightly textured surface provides a superior grip. The Weight clearly labeled on both sides.
Its ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more. The personal tool to help you develop core strength, balance, and coordination.



"""

In [44]:
response, latency = call_bedrock('amazon.titan-tg1-large', prompt_data)
print(response, "\n\n", "Inference time:", latency)

This text is about a weighted medicine ball. The ball has a lightly textured surface to provide a superior grip, and its weight is clearly labeled on both sides. It is ideal for classic medicine ball workouts and can help develop core strength, balance, and coordination. 

 Inference time: 0:00:01.951318


In [45]:
prompt_data = """Human: Provide a summary of the text inside the <metadata></metadata> XML tags.

<metadata>
Weighted medicine ball for upper, lower, and full body exercises. The ball has a lightly textured surface provides a superior grip. The Weight clearly labeled on both sides.
Its ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more. The personal tool to help you develop core strength, balance, and coordination.
</metadata>

Asistant:
"""

In [46]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)


<summary> 
- Weighted medicine ball for various exercises.  
- Lightly textured surface for better grip. 
- Weight labeled on both sides.
- Used for medicine ball exercises, including ball toss, twists, squats, sit ups.  
- Helps develop core strength, balance and coordination.
</summary> 

 Inference time: 0:00:02.110315


In [47]:
prompt_data = """
Summarize the following text:
Weighted medicine ball for upper, lower, and full body exercises. The ball has a lightly textured surface provides a superior grip. The Weight clearly labeled on both sides.
Its ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more. The personal tool to help you develop core strength, balance, and coordination.



"""

In [48]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data)
print(response, "\n\n", "Inference time:", latency)

A weighted medicine ball for upper, lower, and full body exercises. 

 Inference time: 0:00:00.829962


In [51]:
prompt_data = """
Here is an ad for a product, please extract the product name and what its use for :

Weighted medicine ball for upper, lower, and full body exercises. The ball has a lightly textured surface provides a superior grip. The Weight clearly labeled on both sides.
Its ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more. The personal tool to help you develop core strength, balance, and coordination.



"""

In [52]:
response, latency = call_bedrock('amazon.titan-tg1-large', prompt_data)
print(response, "\n\n", "Inference time:", latency)

Weighted medicine ball is used for upper, lower, and full body exercises. 

 Inference time: 0:00:01.060757


In [53]:
prompt_data = """Human:
The information about a new product is inside the <metadata></metadata> XML tags, I want you to extract the product name and what its use for

<metadata>
Weighted medicine ball for upper, lower, and full body exercises. The ball has a lightly textured surface provides a superior grip. The Weight clearly labeled on both sides.
Its ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more. The personal tool to help you develop core strength, balance, and coordination.
</metadata>

Asistant:
"""

In [54]:
response, latency = call_bedrock('anthropic.claude-v1', prompt_data)
print(response, "\n\n", "Inference time:", latency)


Product name: Weighted medicine ball
Use: For upper, lower, and full body exercises. Ideal for medicine ball workouts that help develop core strength, balance and coordination. 

 Inference time: 0:00:01.341715


In [57]:
prompt_data = """
Here is an description of product, extract the product name and what its use for :

Weighted medicine ball for upper, lower, and full body exercises. The ball has a lightly textured surface provides a superior grip. The Weight clearly labeled on both sides.
Its ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more. The personal tool to help you develop core strength, balance, and coordination.



"""

In [58]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data)
print(response, "\n\n", "Inference time:", latency)

The product name is "Weighted Medicine Ball" and it is used for upper, lower, and full body 

 Inference time: 0:00:00.596962


In [75]:
prompt_data ="""Generate a list of reasons why customers should have a Medicine Ball at home using the following data in 2 sentences: Weighted medicine ball for upper, lower, and full body exercises
The ball has a lightly textured surface provides a superior grip. Weight clearly labeled on both sides. Ideal for classic medicine ball workouts, including ball toss, weighted twists, squats, sit ups, and more. The person tool to help you develop core strength, balance, and coordination."""

In [76]:
response, latency = call_bedrock('ai21.j2-jumbo-instruct', prompt_data)
print(response, "\n\n", "Inference time:", latency)


A medicine ball is a versatile piece of equipment that can be used for a variety of exercises, including upper, lower, and full body exercises. The ball has a lightly textured surface that provides a good grip, and the weight is clearly labeled on both sides. 

 Inference time: 0:00:01.143750
